## 明确目的

RFM模型(衡量客户价值)：对比分析不同用户群体在时间，地区等维度下交易量，交易金额指标，并根据分析结果提出优化建议

R:客户最近一次消费时间

F:消费的频次(单位时间内消费了多少次)

M:消费的金额(单位时间内总消费金额)


In [1]:
#忽略报警信息
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os
import plotly as py

In [ ]:
#初始化pltly让plotly在notebook中能用
py.offline.init_notebook_mode()

## 获取数据

In [ ]:
df = pd.read_csv('数据目录')

In [ ]:
df.shape

In [ ]:
df.info

## 数据清洗

In [ ]:
#1. 案列统计缺失率
df.apply(lambda x: sum(x.isnull()/len(x),axis=0))

In [ ]:
#2. 删除掉不需要的字段列,axis=1表示列的方向进行迭代删除
df.drop(['Description'],axis=1,inplace=Ture)

In [ ]:
#3.缺失值处理,将cUSTOMERID这一列的缺失值填充为U
df['CustomerID'] = df['CustomerID'].fillna('U')

In [ ]:
#对订单日期进行拆分，增加年月日日期４个字段,以及增加字段：合计购买
df['amount'] = df['Quantity'] * df['UnitPrice']

In [ ]:
#对dtype为str的数据，可以进行字符串操作
df['dates'] = [x.split(' ')[0] for x in df['InvoiceDate']]

In [ ]:
df['time'] = [x.split(' ')[1] for x in df['InvoiceDate']]

In [ ]:
df[['date','time']]

In [ ]:
df.drop(['InvoiceDate'],axis=1,inplace=True)

In [ ]:
df['year'] = [x.split('/')[2] for x in df['date']]
df['month'] = [x.split('/')[0] for x in df['date']]
df['day'] = [x.split('/')[1] for x in df['date']]

In [ ]:
#将字符型日期转为datetime64
df['date'] = pd.to_datetime(df['date'])

In [ ]:
#删除重复值
df = df.drop_duplicates()

In [ ]:
#描述性统计
df.describe()

## 数据分析

In [ ]:
df1 = df.loc[df['Quantity']<=0]

In [ ]:
df1 = pd.pivot(df1,
               index='year',  #以年为行标签
               columns='month',#以月为列标签
               values='amount',   #对字段amount进行统计
              aggfunc={'amount':np.sum},
               margins=False    #聚合函数

退货率 = 退货金额/合计金额
注：退货金额表示为负数

In [ ]:
np.abs(tt/df1)

### 分析客户的rfm模型

#### R：每个客户最近一次的消费时间

In [ ]:
R_value = df2.groupby('CustomerID')['date'].max()
R_value

In [ ]:
R_value.describe()

In [ ]:
#计算距离每个客户最后一天消费的天数
#datetime64日期类型支持日期运算
#dt.days获取运算后的天数
R_Value = (df2['date'].max() - R_value).dt.days

#### f:#计算客户的消费频次

In [ ]:
#nunique()返回去重后的定案编号的个数
F_value = df.groupby('CustomerID')['InvoiceNo'].nunique()

#### M:计算每个客户的消费总金额

In [ ]:
M_value = df2.groupby('CustomerID')['amount'].sum()

#### 对用户进行分级(数据离散化)

In [ ]:
#对用户最近次购买的天数进行分段：最近消费30天,90天,180天,360天内的客户
R_bins = [0,30,90,180,360,720]
#对用户的消费频次进行分段
F_bins = [1,2,5,10,20,5000]
#对用户的消费总金额进行分段
M_bins = [0,500,2000,5000,10000,200000]

In [ ]:
R_score = pd.cut(R_value,R_bins,labels=[5,4,3,2,1],right=False)

In [ ]:
F_score = pd.cut(F_value,F_bins,labels=[1,2,3,4,5],right=False)

In [ ]:
M_score = pd.cut(M_value,M_bins,labels=[1,2,3,4,5],right=False)

In [ ]:
rfm = pd.concat([R_score,F_score,M_score],axis=1)

In [ ]:
#给rfm的列级索引附上标签
rfm.rename(columns={'date':'R_score','invoice':'F_score','amount':'M_score'})

In [ ]:
#查看字段的dtype
rfm.info()
#返现数据类型为category,为了方便计算，需要转为数值型
for i in ['R_score','F_core','M_score']:
    rfm[i] = rfm[i].astype(float)

In [ ]:
frm.describe()

In [ ]:
#向rfm中添加字段，R_score>3.82的为高价值客户
rfm['R'] = np.where(rfm['R_score'>3.82,'高','低'])
#F_score>2.03的为高价值客户
rfm['F'] = np.where(rfm['R_score']>2.03,'高','低')
#M_score>2.03的为高价值客户
rfm['M'] = np.where(rfm['M_score']>2.03,'高','低')

In [ ]:
#添加价值变量value
rfm['value'] = rfm['R'].str[:] + rfm['F'].str[:] + rfm['M'].str[:]

In [ ]:
rfm['value'] = rfm['value'].str.strip()

In [ ]:
def trans_value(x):
    if x == '高高高':
        return '重要价值客户'
    elif x == '高低高':
        return '重要发展客户'
    elif x == '低高高':
        return '重要保持客户'
    elif x == '低低高':
        return '重要挽留客户'
    elif x == '高高低':   #消费不高
        return '一般价值客户'
    elif x == '高高低':   
        return '一般发展客户'
    elif x == '低高低':
        return '一般保持客户'
    else:
        return '一般挽留客户'

In [ ]:
#添加用户等级字段
rfm['用户等级'] = rfm['value'].apply(trans_value)

In [ ]:
rfm['用户等级'].value_counts()

In [1]:
#使用plotly进行可视化
from plotly import *
#准备图轨数据
trace_basic = [graph_objs.Bar(x=rfm['用户等级'].value_counts().index,
                             y=rfm['用户等级'].value_counts().values,
                             marker=dict(color='orange'),
                             opacity=0.5)]

#设置布局
layout = graph_objs.Layout(title='用户等级情况',
                          xaxis=dict(title='用户重要度'))

#将图轨数据和布局集成到一起
figure_basic = graph_objs.Figure(data=trace_basic,
                                layout=layout)

#绘图，绘制客户分类占比饼图

NameError: name 'graphic' is not defined

#### 结论和建议

从产品，渠道，价格，促销四个方面分析找到原因进行改进

精准营销：
重要价值客户和重要发展客户占客户人群最多
重要发展客户，由于最近的购物较少，及时推送公司活动挽回客户
其次为一般发展客户和一般挽留客户
一般发展客户，获取客户的详细数据用户画像，了解客户的消费需求，及时推送公司活动
对于重要保持客户和重要挽留客户，赠送优惠券或推送折扣，增加用户活跃